In [10]:
!pip install selenium
!pip install webdriver_manager



In [14]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Function to set up the driver
def setup_driver():
    options = Options()
    options.headless = True  # Set to False if you want to see the browser window
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver


def get_news(query, limit=50, country_code='us'):
    driver = setup_driver()
    formatted_query = '+'.join(query.split())
    news_url = f"https://www.google.com/search?q={formatted_query}&tbm=nws&source=lnt&tbs=sbd:1&gl={country_code}"
    driver.get(news_url)
    wait = WebDriverWait(driver, 10)
    news_list = []

    try:
        while len(news_list) < limit:
            time.sleep(2)
            news_boxes = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="rso"]/div/div/div/div/div/a')))
            for box in news_boxes:
                if len(news_list) >= limit:
                    break
                title_element = box.find_element(By.XPATH, './/div[@class="n0jPhd ynAwRc MBeuO nDgy9d"]')
                title = title_element.text
                link = box.get_attribute('href')
                news_list.append((title, link))
                
            next_button = driver.find_element(By.XPATH, '//a[@id="pnnext"]')
            next_button.click()
    except Exception as e:
        print(f"Error during news collection or navigation: {e}")
    finally:
        driver.quit()

    # Exporting to CSV
    with open('news_list.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'URL'])  # Writing the headers
        writer.writerows(news_list)

    return news_list

# User input for country-specific search
country_code = input("Enter the country code (e.g., 'us' for USA, 'uk' for UK): ")
query = input("Enter your search query for Google News: ")
news_query=query+" +tourism +travel "
news_articles = get_news(news_query, country_code=country_code)

for idx, (title, link) in enumerate(news_articles, 1):
    print(f"{idx}. {title} - {link}")


Enter the country code (e.g., 'us' for USA, 'uk' for UK): us
Enter your search query for Google News: mumbai
1. Top tourism companies in India - https://housing.com/news/top-tourism-companies-in-india/
2. India's most Instagrammed tourist hotspots in 2024 - https://www.thenationalnews.com/travel/2024/04/16/top-tourist-places-india/
3. Travel influencer Tara Katims is slammed for going on slum tour of Mumbai and saying it's 'her favourite part - https://www.dailymail.co.uk/femail/article-13206985/american-influencer-slammed-india-dharavi-slums.html
4. From Ed Sheeran to Taylor Swift, more Indians travel for the live music experience - https://www.thehindu.com/entertainment/from-ed-sheeran-to-taylor-swift-more-indians-travel-for-the-live-music-experience/article67935123.ece
5. Navi Mumbai Airport to Spark Tourism Boom With Plans For Tourist Zone - https://www.travelandleisureasia.com/in/news/navi-mumbai-airport-eateries-parks-hotels-to-open-in-the-vicinity/
6. Navi Mumbai Airport to get 

In [12]:
pip install pysentimiento

Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
from pysentimiento import create_analyzer

# Create a sentiment analyzer for English
analyzer = create_analyzer(task="sentiment", lang="en")

def analyze_titles(csv_path):
    # Read the news titles from CSV
    with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Skip the header
        news_data = [row for row in reader]

    # Analyze sentiment and append results
    analyzed_data = []
    for row in news_data:
        if len(row) < 2:
            continue  # Skip rows that don't have at least two columns
        title = row[0]  # Assuming the first column is the title
        url = row[1]  # Assuming the second column is the URL
        analysis = analyzer.predict(title)
        sentiment = max(analysis.probas, key=analysis.probas.get)
        score = analysis.probas[sentiment]
        analyzed_data.append(row + [sentiment, score])  # Append sentiment data to the existing data

    # Write the updated data back to the CSV
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers + ['Sentiment', 'Score'])  # Append new columns to the header
        writer.writerows(analyzed_data)
        
print("Sentiment Analysis done for all news.")

# Path to your CSV file
csv_path = 'shimla.csv'
analyze_titles(csv_path)


Sentiment Analysis done for all news.


In [25]:
emotion_analyzer = create_analyzer(task="emotion", lang="en")
emotion_analyzer.predict("Good News: Ahmedabad-Delhi Bullet Train to Cut Travel Time to 3.5 Hours.")


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

C:\Users\Ummeed\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ummeed\.cache\huggingface\hub\models--finiteautomata--bertweet-base-emotion-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

AnalyzerOutput(output=others, probas={others: 0.925, joy: 0.065, surprise: 0.005, fear: 0.002, disgust: 0.001, anger: 0.001, sadness: 0.001})